In [1]:
using Revise 

module_path = "/home/artur/BondPricing/Julia/modules/"
include(string(module_path, "/", "ModelObj.jl"))
modnames = ["AnalyticFunctions", "BondPrInterp", "EqFinDiff"]
for modl in modnames
    include(string(joinpath(module_path, modl), "/", modl, ".jl"))
end

In [2]:
pardict = Dict("V0" => 100, 
               "vbl" => 60, 
               "vbh" => 62, 
                "m" => 1.,
                "c" => 6.39,
                "p" => 61.68,
                "sigmal" => .25,
                "sigmah" => .25,
                "r" => .08,
                "gross_delta" => .02,
                "iota" => .0,
                "xi" => 1.,
                "kappa" => .015,
                "alpha" => .6,
                "pi" => .27,
                "lambda" => .3)

svm = ModelObj.firm_constructor(pardict)

Main.ModelObj.Firm
  m: Float32 1.0f0
  c: Float32 6.39f0
  p: Float32 61.68f0
  vbl: Float32 60.0f0
  vbh: Float32 62.0f0
  pm: Main.ModelObj.FirmParams


In [3]:
Int8(3.)

3

In [12]:
bprd = @time BondPrInterp.fd_interp_inputs(svm, vtN=10, vbhlN=5)

 43.038660 seconds (20.85 M allocations: 1.832 GiB, 2.05% gc time)


Dict{String,AbstractArray{Float64,N} where N} with 7 entries:
  "vtgrid"   => 0.08:0.08:0.8
  "f12"      => [0.101968 0.0900789 … 0.0356983 0.0179562; 0.164741 0.151849 … …
  "f13"      => [0.00451994 0.016928 … 0.035561 0.0178967; 0.00400585 0.0173554…
  "f22"      => [1.79793e-7, 8.60424e-7, 3.64161e-6, 1.40637e-5, 4.96701e-5, 0.…
  "f21"      => [1.03711e-7, 4.98978e-7, 2.12564e-6, 8.27191e-6, 2.94771e-5, 9.…
  "f11"      => [0.112906, 0.178345, 0.214121, 0.23248, 0.241262, 0.245118, 0.2…
  "vbhlgrid" => 0.75:0.125:1.25

In [72]:

function input2csv(bprd, fpath)
    for key in keys(bprd)
        println(key)
        if size(bprd[key], 2) == 1
            df = DataFrame(key = bprd[string(key)])
            header = key
        else
            df = convert(DataFrame, bprd[key])
            header = bprd["vbhlgrid"]
        end
        CSV.write(string(fpath, "/", key), df, colnames=header)
    end
end
    

input2csv (generic function with 1 method)

In [73]:
fpath = "/home/artur/BondPricing/Python/JuliaFiles"
input2csv(bprd, fpath)

vtgrid
f12
f13
f22
f21
f11
vbhlgrid


In [74]:
size(bprd["f12"], 2)

5

In [3]:
using JLD, HDF5

In [4]:
fpath = "/home/artur/BondPricing/Python/JuliaFiles"
fname = "test.h5"
myfile = string(fpath,"/", fname)

"/home/artur/BondPricing/Python/JuliaFiles/test.h5"

In [10]:
save(string(fpath,"/", "test.jld"), "bprd", bprd)

In [5]:
bprd2 = load(string(fpath,"/", "test.jld"))

bprd

Dict{String,Any} with 1 entry:
  "bprd" => Dict{String,AbstractArray{Float64,N} where N}("vtgrid"=>0.08:0.08:0…

In [ ]:
h5open(myfile, "w") do file
    write(file, "vtgrid", bprd["vtgrid"])  # alternatively, say "@write file A"
end

In [14]:
h5write(string(fpath,"/", fname), "jVars/vtgrid", bprd["vtgrid"])

MethodError: MethodError: no method matching strides(::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}})
Closest candidates are:
  strides(!Matched::SubArray) at subarray.jl:264
  strides(!Matched::Base.CodeUnits) at strings/basic.jl:696
  strides(!Matched::PermutedDimsArray{T,N,perm,iperm,AA} where AA<:AbstractArray where iperm) where {T, N, perm} at permuteddimsarray.jl:62
  ...

In [15]:
bprd

Dict{String,AbstractArray{Float64,N} where N} with 7 entries:
  "vtgrid"   => 0.08:0.08:0.8
  "f12"      => [0.101968 0.0900789 … 0.0356983 0.0179562; 0.164741 0.151849 … …
  "f13"      => [0.00451994 0.016928 … 0.035561 0.0178967; 0.00400585 0.0173554…
  "f22"      => [1.79793e-7, 8.60424e-7, 3.64161e-6, 1.40637e-5, 4.96701e-5, 0.…
  "f21"      => [1.03711e-7, 4.98978e-7, 2.12564e-6, 8.27191e-6, 2.94771e-5, 9.…
  "f11"      => [0.112906, 0.178345, 0.214121, 0.23248, 0.241262, 0.245118, 0.2…
  "vbhlgrid" => 0.75:0.125:1.25

In [17]:
using DataFrames, Feather

In [30]:
bprd["f12"]

10×5 Array{Float64,2}:
 0.101968  0.0900789  0.0657757  0.0356983  0.0179562
 0.164741  0.151849   0.121613   0.0756957  0.0397393
 0.200588  0.190924   0.164749   0.118549   0.070012 
 0.219529  0.213609   0.194999   0.157209   0.108649 
 0.228751  0.225633   0.214213   0.187569   0.147207 
 0.232839  0.231396   0.225221   0.208652   0.179477 
 0.234366  0.233773   0.2308     0.221615   0.20293  
 0.234518  0.2343     0.233019   0.228453   0.217758 
 0.23353   0.233458   0.232962   0.230921   0.225424 
 0.23088   0.230859   0.230686   0.229864   0.227321 

In [63]:
df = DataFrame(f11 = bprd["f11"])

,f11
,Float64
1,0.112906
2,0.178345
3,0.214121
4,0.23248
5,0.241262
6,0.245118
7,0.246543
8,0.24665
9,0.245625


In [44]:
df = convert(DataFrame, bprd["f12"]')

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.101968,0.164741,0.200588,0.219529,0.228751,0.232839,0.234366,0.234518,0.23353,0.23088
2,0.0900789,0.151849,0.190924,0.213609,0.225633,0.231396,0.233773,0.2343,0.233458,0.230859
3,0.0657757,0.121613,0.164749,0.194999,0.214213,0.225221,0.2308,0.233019,0.232962,0.230686
4,0.0356983,0.0756957,0.118549,0.157209,0.187569,0.208652,0.221615,0.228453,0.230921,0.229864
5,0.0179562,0.0397393,0.070012,0.108649,0.147207,0.179477,0.20293,0.217758,0.225424,0.227321


In [43]:
using CSV
fpath = "/home/artur/BondPricing/Python/JuliaFiles"
output = string(fpath,"/", "test")

CSV.write(output, df)

"/home/artur/BondPricing/Python/JuliaFiles/test"

In [40]:
df.DataFrameColumns

KeyError: KeyError: key :DataFrameColumns not found

In [38]:
 DataFrame(i = 1:10, x = rand(10), y = rand(["a", "b", "c"], 10))

,i,x,y
,Int64,Float64,String
1,1,0.304232,a
2,2,0.533728,c
3,3,0.983624,c
4,4,0.0860258,c
5,5,0.0425041,a
6,6,0.739395,b
7,7,0.791789,b
8,8,0.762882,b
9,9,0.489373,a


In [33]:
df.colindex

KeyError: KeyError: key :colindex not found

In [36]:
names!(df, [vbhl for vbhl in bprd["vbhlgrid"]], makeunique=true)

MethodError: MethodError: no method matching names!(::DataFrames.Index, ::Array{Float64,1}; makeunique=true)
Closest candidates are:
  names!(::DataFrames.Index, !Matched::Array{Symbol,1}; makeunique) at /home/artur/.julia/packages/DataFrames/KUWId/src/other/index.jl:27
  names!(!Matched::AbstractDataFrame, ::Any; makeunique) at /home/artur/.julia/packages/DataFrames/KUWId/src/abstractdataframe/abstractdataframe.jl:120

In [44]:
fdf = @time BondPrInterp.fd_input_interp(bprd)

  1.823133 seconds (6.86 M allocations: 323.489 MiB, 6.02% gc time)


Dict{String,Interpolations.ScaledInterpolation{Float64,N,ITPT,Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},RT} where RT where ITPT where N} with 5 entries:
  "f12" => [0.101968 0.0900789 … 0.0356983 0.0179562; 0.164741 0.151849 … 0.075…
  "f13" => [0.00451994 0.016928 … 0.035561 0.0178967; 0.00400585 0.0173554 … 0.…
  "f22" => [1.79793e-7, 8.60424e-7, 3.64161e-6, 1.40637e-5, 4.96701e-5, 0.00016…
  "f21" => [1.03711e-7, 4.98978e-7, 2.12564e-6, 8.27191e-6, 2.94771e-5, 9.63561…
  "f11" => [0.112906, 0.178345, 0.214121, 0.23248, 0.241262, 0.245118, 0.246543…

In [35]:
bmat = @time BondPrInterp.mat_interp_inputs(svm, ttmN=10, vbhlN=5)

 38.814138 seconds (3.04 M allocations: 1.279 GiB, 0.86% gc time)


Dict{String,AbstractArray{Float64,N} where N} with 7 entries:
  "f12"      => [0.0262371 0.00389506 … 1.3898e-7 3.41844e-11; 0.0471068 0.0112…
  "f13"      => [5.43319e-74 4.25071e-71 … 5.13405e-71 5.50795e-73; 1.48019e-38…
  "f22"      => [0.00321236 0.0261217 … 0.0301071 0.0301072; 0.0109343 0.048565…
  "f21"      => [0.003093 0.0255107 … 0.0293551 0.0293552; 0.0101985 0.0463545 …
  "f11"      => [0.026368 0.00390794 … 1.39096e-7 3.42014e-11; 0.0475884 0.0113…
  "vbhlgrid" => 0.75:0.125:1.25
  "ttmgrid"  => 0.1:0.1:1.0

In [197]:
matf = @time BondPrInterp.mat_input_interp(bmat)

here
  0.050691 seconds (38.63 k allocations: 2.058 MiB)


Dict{String,ScaledInterpolation{Float64,N,ITPT,BSpline{Cubic{Line{OnGrid}}},RT} where RT where ITPT where N} with 5 entries:
  "f12" => [0.0292746 0.0292746 … 0.0292743 0.0292112; 0.0571376 0.0571376 … 0.…
  "f13" => [1.76422e-23 8.28171e-16 … 3.11806e-7 5.87569e-5; 4.84984e-13 3.1852…
  "f22" => [4.76679e-24, 7.47474e-14, 2.83146e-10, 2.12383e-8, 3.18091e-7, 2.08…
  "f21" => [4.49376e-24, 6.66498e-14, 2.3916e-10, 1.70203e-8, 2.4219e-7, 1.5104…
  "f11" => [0.0294146, 0.0576888, 0.0848608, 0.110945, 0.135936, 0.159823, 0.18…

In [46]:
using JLD

┌ Info: Precompiling JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 7094110003843402 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module Compat with build ID 7094110003843402 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module Compat with build ID 7094110003843402 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module Compat with build ID 7094110003843402 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is importe

In [47]:
save("/home/artur/BondPricing/Python/JupyterNotebooks/test.jld", "fdf", fdf)

In [48]:
fdf2 = load("/home/artur/BondPricing/Python/JupyterNotebooks/test.jld")

┌ Warning: type JLD.AssociativeWrapper{Core.String,@where(@where(@where(Interpolations.ScaledInterpolation{Core.Float64,N,ITPT,Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},RT},RT),ITPT),N),Base.Dict{Core.String,@where(@where(@where(Interpolations.ScaledInterpolation{Core.Float64,N,ITPT,Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},RT},RT),ITPT),N)}} not present in workspace; reconstructing
└ @ JLD /home/artur/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type @where(@where(@where(Interpolations.ScaledInterpolation{Core.Float64,N,ITPT,Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},RT},RT),ITPT),N) not present in workspace; interpreting array as Array{Any}
└ @ JLD /home/artur/.julia/packages/JLD/1BoSz/src/JLD.jl:486
┌ Warning: type Interpolations.ScaledInterpolation{Core.Float64,2,Interpolations.BSplineInterpolation{Core.Float64,2,OffsetArrays.Offse

Dict{String,Any} with 1 entry:
  "fdf" => ##JLD.AssociativeWrapper{Core.String,@where(@where(@where(Interpolat…

In [51]:
fdf["f11"](.3)

0.22895551214686474

In [121]:
using Interpolations

In [36]:
vtmax=.8 
vtN=20
_, vtgrid=BondPrInterp.grid_creator(0.0, vtmax, vtN)
vbhlmin=.75
vbhlmax=1.25
vbhlN=5
_, vbhlgrid=BondPrInterp.grid_creator(vbhlmin, vbhlmax, vbhlN)
vmax=1.2
vN=10^3
uN=10^3

1000

In [64]:
ttm_max=svm.m

dv, vgrid = BondPrInterp.grid_creator(0.0, vmax, vN)
du, ugrid = BondPrInterp.grid_creator(0.0, ttm_max, uN) 
vt = log(100/63)


tmp = BondPrInterp.f11v_int(vt, ugrid, vgrid, svm.pm.lambda, svm.pm.sigmal, 
                            svm.pm.r, svm.pm.gross_delta, svm.pm.xi, svm.pm.kappa)

tmp

0.24455625310486812

In [65]:
tmp = BondPrInterp.f21_int(.5, vmax, svm.pm.lambda, svm.pm.sigmal, 
                svm.pm.r, svm.pm.gross_delta, svm.pm.xi, svm.pm.kappa; 
		ttm=1.0, N=10^2, ugrid=nothing)

0.00012785480104962

In [68]:
ttmN = 10
_, ttmgrid = BondPrInterp.grid_creator(0.0, ttm_max, ttmN)

BondPrInterp.f11_mat_inputs(vt, ttmgrid, 
                            svm.pm.lambda, svm.pm.sigmal, 
                            svm.pm.r, svm.pm.gross_delta, svm.pm.xi, svm.pm.kappa;
                            vmax=vmax)
		


9-element Array{Float64,1}:
 0.028304130733514957
 0.05550619990435978 
 0.08162072228358676 
 0.10664083573910031 
 0.13055831362700548 
 0.1533751903636168  
 0.17510564889255914 
 0.19577392146281192 
 0.2154114567775562  